In [ ]:
import requests
from bs4 import BeautifulSoup
import json

class NewsAPI:
    BASE_URL = "https://news.google.com/rss"

    def __init__(self):
        self.language = "en"
        self.country = "US"
        self.limit = 10

    def set_query_options(self, language=None, country=None, limit=None):
        if language:
            self.language = language
        if country:
            self.country = country
        if limit:
            self.limit = limit

    def get_top_news(self):
        url = f"{self.BASE_URL}/headlines/section/topic/WORLD?hl={self.language}&gl={self.country}&ceid={self.country}:{self.language}"
        return self._fetch_news(url)

    def get_location_news(self, location):
        url = f"{self.BASE_URL}/headlines/section/geo/{location}?hl={self.language}&gl={self.country}&ceid={self.country}:{self.language}"
        return self._fetch_news(url)

    def get_topic_news(self, topic):
        url = f"{self.BASE_URL}/topics/{topic}?hl={self.language}&gl={self.country}&ceid={self.country}:{self.language}"
        return self._fetch_news(url)

    def search_news(self, query):
        url = f"{self.BASE_URL}/search?q={query}&hl={self.language}&gl={self.country}&ceid={self.country}:{self.language}"
        return self._fetch_news(url)

    def _fetch_news(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'xml')
        items = soup.find_all('item', limit=self.limit)

        news_list = []
        for item in items:
            news = {
                'title': item.title.text,
                'link': item.link.text,
                'pubDate': item.pubDate.text,
                'description': item.description.text
            }
            news_list.append(news)

        return news_list

    @staticmethod
    def fetch_news_content(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        content = ' '.join([p.text for p in paragraphs])
        return content

# 사용 예시
if __name__ == "__main__":
    api = NewsAPI()

    # 쿼리 옵션 설정
    api.set_query_options(language="ko", country="KR", limit=5)

    # 상위 뉴스 가져오기
    top_news = api.get_top_news()
    print("Top News:")
    for news in top_news:
        print(f"- {news['title']}")

    # 특정 위치의 뉴스 가져오기
    location_news = api.get_location_news("SEOUL")
    print("\nSeoul News:")
    for news in location_news:
        print(f"- {news['title']}")

    # 특정 주제의 뉴스 가져오기
    topic_news = api.get_topic_news("TECHNOLOGY")
    print("\nTechnology News:")
    for news in topic_news:
        print(f"- {news['title']}")

    # 뉴스 검색하기
    search_news = api.search_news("Python programming")
    print("\nSearch Results for 'Python programming':")
    for news in search_news:
        print(f"- {news['title']}")

    # 뉴스 내용 가져오기
    if search_news:
        content = NewsAPI.fetch_news_content(search_news[0]['link'])
        print(f"\nContent of first search result:\n{content[:500]}...")